In [1]:
import findspark

In [2]:
import re

In [3]:
findspark.init('/home/cse587/spark-2.4.0-bin-hadoop2.7')

In [4]:
import pyspark

In [5]:
from pyspark.sql import *
from pyspark.sql.functions import *

In [6]:
sc = pyspark.SparkContext()

In [7]:
spark = SparkSession \
    .builder \
    .appName("PA3") \
    .config("spark.some.config.option","some-value") \
    .getOrCreate()

In [8]:
import pandas as pd
df = pd.read_csv("train.csv")

In [9]:
df = spark.createDataFrame(df)

In [10]:
df1 =df.withColumn('plot',lower(regexp_replace(df['plot'],"[^a-zA-Z ]","")))

In [11]:
from nltk.corpus import stopwords
import nltk 
nltk.download('stopwords')
df1.take(1)

[nltk_data] Downloading package stopwords to /home/cse587/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


[Row(movie_id=23890098, movie_name='Taxi Blues', plot='shlykov a hardworking taxi driver and lyosha a saxophonist develop a bizarre lovehate relationship and despite their prejudices realize they arent so different after all', genre="['World cinema', 'Drama']")]

In [37]:
stop = stopwords.words('english')
df1.printSchema()


root
 |-- movie_id: long (nullable = true)
 |-- movie_name: string (nullable = true)
 |-- plot: string (nullable = true)
 |-- genre: string (nullable = true)



In [13]:
df1 = df1.dropna()

In [14]:
df2 = df1.select('plot')

In [45]:
rd = df2.rdd.flatMap(list)
data = df2.rdd.map(list)
data = data.map(lambda x: x[0].split())
def dd(x):
    to_remove = []
    for word in x:
        if word not in stop:
            continue
        else:
            to_remove.append(word)
    for word in to_remove:
        x.remove(word)
    return x
data = data.map(dd)
data.take(1)


[['shlykov',
  'hardworking',
  'taxi',
  'driver',
  'lyosha',
  'saxophonist',
  'develop',
  'bizarre',
  'lovehate',
  'relationship',
  'despite',
  'prejudices',
  'realize',
  'arent',
  'different']]

In [44]:
rd1 = rd.flatMap(lambda x: x.split())
rd1 = rd1.filter(lambda x: x not in stop)

In [17]:
m = rd1.map(lambda x: (x,1))

In [18]:
wordCount = m.reduceByKey(lambda a,b: a+b)


In [19]:
common_words = wordCount.filter(lambda x: x[1] > 200)

In [20]:
common_words.count()

4312

In [21]:
features = common_words.keys()

In [22]:
from pyspark .mllib.linalg import SparseVector

In [23]:
from collections import Counter
def remove_uncommon_words(x):
    for word in x:
        if word in features:
            continue
        else:
            x.remove(word)
    return x


In [148]:
features= features.collect()

AttributeError: 'list' object has no attribute 'collect'

In [149]:
data = data.map(remove_uncommon_words)

In [150]:
vocab_map = data.flatMap(lambda token: token).distinct() \
    .zipWithIndex().collectAsMap()

In [151]:
v_map = sc.broadcast(vocab_map)
vocab_size = sc.broadcast(len(vocab_map))

In [152]:
tdm = data \
    .map(Counter) \
    .map(lambda counts: {v_map.value[token]: float(counts[token]) for token in counts})\
    .map(lambda index_counts: SparseVector(vocab_size.value, index_counts))

In [153]:
labels = df1.select('genre')

In [154]:
labels= labels.rdd.flatMap(lambda x:x)


labels.take(2)

["['World cinema', 'Drama']",
 "['Action/Adventure', 'Action', 'Science Fiction', 'Drama']"]

In [155]:
labels.take(3)

["['World cinema', 'Drama']",
 "['Action/Adventure', 'Action', 'Science Fiction', 'Drama']",
 "['Musical', 'Action', 'Drama']"]

In [156]:
tdm.first()

SparseVector(83408, {0: 1.0, 1: 1.0, 2: 1.0, 3: 1.0, 4: 1.0, 41752: 1.0, 41753: 1.0, 41754: 1.0, 41755: 1.0, 41756: 1.0})

In [157]:
data.take(1)

[['hardworking',
  'taxi',
  'driver',
  'saxophonist',
  'develop',
  'bizarre',
  'relationship',
  'despite',
  'realize',
  'different']]

In [158]:
movie_ids = df.select('movie_id')
movie_ids = movie_ids.rdd
movie_ids = movie_ids.flatMap(lambda x:x)
movie_ids = movie_ids.collect()


In [179]:
keylist = pd.read_csv('mapping.csv')
keylist.rename(columns={'Unnamed: 0': 'i'}, inplace=True)
keylist = spark.createDataFrame(keylist)
k = keylist.select('0')
k1 = keylist.select('i')
k = k.rdd.flatMap(lambda x:x)
k1 = k1.rdd.flatMap(lambda x:x)
genre_list = k.zip(k1)
# genre_list.collect()

In [180]:
import numpy as np

# n = tdm.count()
gl = genre_list.collect()

In [181]:
i=0

def comb(x):
    a = np.zeros(20)
    for g in x[i]:
        for t in gl:
            if t[0] == g:
                a[t[1]]=1 
    i+=1
    return a

genre_list = labels.map(comb)
    

In [183]:
genre_list.take(1)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 93.0 failed 1 times, most recent failure: Lost task 0.0 in stage 93.0 (TID 128, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/cse587/spark-2.4.0-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/worker.py", line 372, in main
    process()
  File "/home/cse587/spark-2.4.0-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/worker.py", line 367, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/home/cse587/spark-2.4.0-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/serializers.py", line 390, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/home/cse587/spark-2.4.0-bin-hadoop2.7/python/pyspark/rdd.py", line 1354, in takeUpToNumLeft
    yield next(iterator)
  File "/home/cse587/spark-2.4.0-bin-h

# Part 2: TF-IDF

In [ ]:
 from pyspark.mllib.feature import HashingTF, IDF